## 이전 코드

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

from typing import Literal

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

# API 키 정보 로드
load_dotenv()
# 체크포인트 저장을 위한 메모리 객체 초기화
memory = MemorySaver()


# 웹 검색 기능을 모방하는 도구 함수 정의
@tool
def search(query: str):
    """Call to surf on the web."""
    return "웹 검색 결과: LangGraph 한글 튜토리얼은 https://wikidocs.net/233785 에서 확인할 수 있습니다."


# 도구 목록 생성 및 도구 노드 초기화
tools = [search]
tool_node = ToolNode(tools)

# 모델 초기화 및 도구 바인딩
model = ChatOpenAI(model_name="gpt-4o-mini")
bound_model = model.bind_tools(tools)


# # 대화 상태에 따른 다음 실행 노드 결정 함수
def should_continue(state: MessagesState):
    last_message = state["messages"][-1]
    if not last_message.tool_calls:
        return END
    return "tool"


# LLM 모델 호출 및 응답 처리 함수
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# 상태 기반 워크플로우 그래프 초기화
workflow = StateGraph(MessagesState)

# 에이전트와 액션 노드 추가
workflow.add_node("agent", call_model)
workflow.add_node("tool", tool_node)

# 시작점을 에이전트 노드로 설정
workflow.add_edge(START, "agent")

# 조건부 엣지 설정: 에이전트 노드 이후의 실행 흐름 정의
workflow.add_conditional_edges("agent", should_continue, {"tool": "tool", END: END})

# 도구 실행 후 에이전트로 돌아가는 엣지 추가
workflow.add_edge("tool", "agent")

# 체크포인터가 포함된 최종 실행 가능한 워크플로우 컴파일
app = workflow.compile(checkpointer=memory)

from langchain_core.messages import HumanMessage

# 스레드 ID가 1인 기본 설정 객체 초기화
config = {"configurable": {"thread_id": "1"}}

# 1번째 질문 수행
input_message = HumanMessage(
    content="안녕하세요! 제 이름은 현상입니다. 잘 부탁드립니다."
)

# 스트림 모드로 메시지 처리 및 응답 출력, 마지막 메시지의 상세 정보 표시
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


================================ Human Message =================================

안녕하세요! 제 이름은 현상입니다. 잘 부탁드립니다.
================================== Ai Message ==================================

안녕하세요, 현상님! 만나서 반갑습니다. 어떤 이야기를 나눌까요? 도움이 필요하신 부분이 있으면 말씀해 주세요!


In [4]:
# 후속 질문 수행
input_message = HumanMessage(content="내 이름이 뭐라고요?")

# 스트림 모드로 두 번째 메시지 처리 및 응답 출력
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

내 이름이 뭐라고요?
================================== Ai Message ==================================

당신의 이름은 현상입니다. 맞나요?


모든 입력

In [5]:
messages = app.get_state(config).values["messages"]
for message in messages:
    message.pretty_print()

================================ Human Message =================================

안녕하세요! 제 이름은 현상입니다. 잘 부탁드립니다.
================================== Ai Message ==================================

안녕하세요, 현상님! 반갑습니다. 어떻게 도와드릴까요?
================================ Human Message =================================

내 이름이 뭐라고요?
================================== Ai Message ==================================

당신의 이름은 현상입니다. 맞나요?


## 메시지 삭제

In [6]:
from langchain_core.messages import RemoveMessage

# 메시지 배열의 첫 번째 메시지를 ID 기반으로 제거하고 앱 상태 업데이트
app.update_state(config, {"messages": RemoveMessage(id=messages[0].id)})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1ef9fc12-62dc-6648-8005-7f453af27e53'}}

In [7]:
messages = app.get_state(config).values["messages"]
for message in messages:
    message.pretty_print()

================================== Ai Message ==================================

안녕하세요, 현상님! 반갑습니다. 어떻게 도와드릴까요?
================================ Human Message =================================

내 이름이 뭐라고요?
================================== Ai Message ==================================

당신의 이름은 현상입니다. 맞나요?


In [8]:
app.update_state(config, {"messages": RemoveMessage(id=messages[0].id)})

messages = app.get_state(config).values["messages"]
for message in messages:
    message.pretty_print()

================================ Human Message =================================

내 이름이 뭐라고요?
================================== Ai Message ==================================

당신의 이름은 현상입니다. 맞나요?


## 메시지 삭제 그래프

In [3]:
from langchain_core.messages import RemoveMessage
from langgraph.graph import END


# 메시지 개수가 3개 초과 시 오래된 메시지 삭제 및 최신 메시지만 유지
def delete_messages(state):
    messages = state["messages"]
    if len(messages) > 3:
        return {"messages": [RemoveMessage(id=m.id) for m in messages[:-3]]}


# 메시지 상태에 따른 다음 실행 노드 결정 로직
def should_continue(state: MessagesState) -> Literal["action", "delete_messages"]:
    """Return the next node to execute."""
    last_message = state["messages"][-1]
    # 함수 호출이 없는 경우 메시지 삭제 함수 실행
    if not last_message.tool_calls:
        return "delete_messages"
    # 함수 호출이 있는 경우 액션 실행
    return "action"


# 메시지 상태 기반 워크플로우 그래프 정의
workflow = StateGraph(MessagesState)

# 에이전트와 액션 노드 추가
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# 메시지 삭제 노드 추가
workflow.add_node(delete_messages)

# 시작 노드에서 에이전트 노드로 연결
workflow.add_edge(START, "agent")

# 조건부 엣지 추가를 통한 노드 간 흐름 제어
workflow.add_conditional_edges(
    "agent",
    should_continue,
)

# 액션 노드에서 에이전트 노드로 연결
workflow.add_edge("action", "agent")

# 메시지 삭제 노드에서 종료 노드로 연결
workflow.add_edge("delete_messages", END)

# 메모리 체크포인터를 사용하여 워크플로우 컴파일
app = workflow.compile(checkpointer=memory)


In [4]:
# LangChain 메시지 처리를 위한 HumanMessage 클래스 임포트
from langchain_core.messages import HumanMessage

# 스레드 ID가 포함된 설정 객체 초기화
config = {"configurable": {"thread_id": "3"}}


In [5]:
input_message = HumanMessage(
    content="안녕하세요! 제 이름은 현상입니다. 잘 부탁드립니다."
)
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])

[('human', '안녕하세요! 제 이름은 현상입니다. 잘 부탁드립니다.')]
[('human', '안녕하세요! 제 이름은 현상입니다. 잘 부탁드립니다.'), ('ai', '안녕하세요, 현상님! 만나서 반갑습니다. 무엇을 도와드릴까요?')]


In [6]:
input_message = HumanMessage(
    content="오늘 날씨가 좋죠?"
)
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])

[('human', '안녕하세요! 제 이름은 현상입니다. 잘 부탁드립니다.'), ('ai', '안녕하세요, 현상님! 만나서 반갑습니다. 무엇을 도와드릴까요?'), ('human', '오늘 날씨가 좋죠?')]
[('human', '안녕하세요! 제 이름은 현상입니다. 잘 부탁드립니다.'), ('ai', '안녕하세요, 현상님! 만나서 반갑습니다. 무엇을 도와드릴까요?'), ('human', '오늘 날씨가 좋죠?'), ('ai', '네, 오늘 날씨가 좋다면 정말 기분이 좋을 것 같아요! 하지만 제가 실시간 날씨 정보를 제공할 수는 없어서, 현상님이 계신 지역의 날씨는 확인해 보셔야 할 것 같아요. 오늘 계획하고 계신 일이 있나요?')]
[('ai', '안녕하세요, 현상님! 만나서 반갑습니다. 무엇을 도와드릴까요?'), ('human', '오늘 날씨가 좋죠?'), ('ai', '네, 오늘 날씨가 좋다면 정말 기분이 좋을 것 같아요! 하지만 제가 실시간 날씨 정보를 제공할 수는 없어서, 현상님이 계신 지역의 날씨는 확인해 보셔야 할 것 같아요. 오늘 계획하고 계신 일이 있나요?')]


In [7]:
input_message = HumanMessage(
    content="내 이름이 뭐였지"
)
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])

[('ai', '안녕하세요, 현상님! 만나서 반갑습니다. 무엇을 도와드릴까요?'), ('human', '오늘 날씨가 좋죠?'), ('ai', '네, 오늘 날씨가 좋다면 정말 기분이 좋을 것 같아요! 하지만 제가 실시간 날씨 정보를 제공할 수는 없어서, 현상님이 계신 지역의 날씨는 확인해 보셔야 할 것 같아요. 오늘 계획하고 계신 일이 있나요?'), ('human', '내 이름이 뭐였지')]
[('ai', '안녕하세요, 현상님! 만나서 반갑습니다. 무엇을 도와드릴까요?'), ('human', '오늘 날씨가 좋죠?'), ('ai', '네, 오늘 날씨가 좋다면 정말 기분이 좋을 것 같아요! 하지만 제가 실시간 날씨 정보를 제공할 수는 없어서, 현상님이 계신 지역의 날씨는 확인해 보셔야 할 것 같아요. 오늘 계획하고 계신 일이 있나요?'), ('human', '내 이름이 뭐였지'), ('ai', '죄송하지만, 이전 대화 내용을 기억할 수 없어서 현상님의 이름이 무엇인지 알 수 없습니다. 하지만 현상님이라고 부르면 되겠죠? 다른 궁금한 점이 있으면 말씀해 주세요!')]
[('ai', '네, 오늘 날씨가 좋다면 정말 기분이 좋을 것 같아요! 하지만 제가 실시간 날씨 정보를 제공할 수는 없어서, 현상님이 계신 지역의 날씨는 확인해 보셔야 할 것 같아요. 오늘 계획하고 계신 일이 있나요?'), ('human', '내 이름이 뭐였지'), ('ai', '죄송하지만, 이전 대화 내용을 기억할 수 없어서 현상님의 이름이 무엇인지 알 수 없습니다. 하지만 현상님이라고 부르면 되겠죠? 다른 궁금한 점이 있으면 말씀해 주세요!')]
